In [27]:
'''
-- Based On:
"Building powerful image classification models using very little data"
-- by Francois Chollet
'''
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from keras.models import Model
from keras.optimizers import Adam
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras import backend as K

In [28]:
# dimensions of our images.
img_height, img_width = 200, 200

data_dir = '../input/no-obs-diffs/no_obs_diffs'
epochs = 50
batch_size = 16

input_shape = (img_height, img_width, 1)

In [29]:
inp = Input(shape=input_shape)

x = Conv2D(8, (3, 3), padding='same', activation = 'relu')(inp)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)

x = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)

x = Conv2D(32, (5, 5), padding='same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)

x = Conv2D(64, (5, 5), padding='same', activation = 'relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(4, 4), padding='same')(x)

x = Flatten()(x)
out = Dense(1, activation='sigmoid')(x)

model = Model(inp, out)
optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=["accuracy"])
model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 200, 200, 1)]     0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 200, 200, 8)       80        
_________________________________________________________________
batch_normalization_16 (Batc (None, 200, 200, 8)       32        
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 100, 100, 8)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 100, 100, 16)      3216      
_________________________________________________________________
batch_normalization_17 (Batc (None, 100, 100, 16)      64        
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 50, 50, 16)       

In [31]:
# this is the augmentation configuration we will use for training
datagen = ImageDataGenerator(
    validation_split=0.3,
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='binary',
    subset='training')

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='binary',
    subset='validation')

Found 12457 images belonging to 2 classes.
Found 5338 images belonging to 2 classes.


In [32]:
model.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    verbose=1)

TypeError: fit_generator() got an unexpected keyword argument 'metrics'

In [ ]:
model.save_weights('no_obs_2.h5')